In [1]:
import pandas as pd
import streamlit as st
import joblib

x_numericos = {'latitude': 0, 'longitude': 0, 'accommodates': 0, 'bathrooms': 0, 'bedrooms': 0, 'beds': 0, 'extra_people': 0,
               'minimum_nights': 0, 'ano': 0, 'mes': 0, 'n_amenities': 0, 'host_listings_count': 0}

x_tf = {'host_is_superhost': 0, 'instant_bookable': 0}

x_listas = {'property_type': ['Apartment', 'Bed and breakfast', 'Condominium', 'Guest suite', 'Guesthouse', 'Hostel', 'House', 'Loft', 'Outros', 'Serviced apartment'],
            'room_type': ['Entire home/apt', 'Hotel room', 'Private room', 'Shared room'],
            'cancelation_policy': ['flexible', 'moderate', 'strict', 'strict_14_with_grace_period']
            }

In [ ]:
# Criar interface no Streamlit
st.title("Previsão de Preço no Airbnb")

# Entradas numéricas
st.subheader("Características Numéricas")
inputs_numericos = {}
for key in x_numericos:
    if key in ['latitude', 'longitude']:
        # Configurar 5 casas decimais para latitude e longitude
        inputs_numericos[key] = st.number_input(key, value=float(x_numericos[key]), format="%.5f", step=0.00001)
    elif key == 'extra_people':
        # Configurar 2 casas decimais para extra_people
        inputs_numericos[key] = st.number_input(key, value=float(x_numericos[key]), step=0.01)
    else:
        # Configuração padrão para os demais
        inputs_numericos[key] = st.number_input(key, value=x_numericos[key], step=1)


# Entradas booleanas
st.subheader("Características Booleanas")
inputs_tf = {}
for key in x_tf:
    escolha = st.radio(key, ['Não', 'Sim'], index=x_tf[key])  # Exibir "Não" e "Sim"
    inputs_tf[key] = 1 if escolha == 'Sim' else 0  # Converter de volta para 1 e 0


# Entradas categóricas
st.subheader("Características Categóricas")
inputs_listas = {}
for key, options in x_listas.items():
    inputs_listas[key] = st.selectbox(key, options)

# Botão para processar
if st.button("Prever Valor"):
    # Combinar todas as entradas em um DataFrame
    input_data = {**inputs_numericos, **inputs_tf, **inputs_listas}
    
    # Carregar o modelo
    modelo = joblib.load('modelo.joblib')
    
    # Criar DataFrame com colunas esperadas pelo modelo
    colunas_esperadas = modelo.feature_names_in_
    input_df = pd.DataFrame(columns=colunas_esperadas)
    
    # Preencher os valores para as colunas
    for coluna in colunas_esperadas:
        input_df[coluna] = [input_data.get(coluna, 0)]  # Preencher com 0 se o valor não estiver disponível
    
    st.write("Dados enviados para o modelo:")
    st.write(input_df)

    # Prever o preço
    previsao = modelo.predict(input_df)
    st.write("Preço previsto:", previsao)